## 1. Importing Libraries

In [126]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

## 2.Reading the data

In [127]:
Project_Dir = r'C:\Users\admin\OneDrive\Desktop\Flights_SageMaker_Project'

Data_Dir = 'Data'

In [128]:
os.path.join(Project_Dir, Data_Dir, 'Data_Train.csv')

'C:\\Users\\admin\\OneDrive\\Desktop\\Flights_SageMaker_Project\\Data\\Data_Train.csv'

In [129]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(Project_Dir,Data_Dir,file_name)
    return pd.read_csv(file_path)

In [130]:
flights = get_data('Data_Train')
flights

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR ? DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09-06-2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,10-06-2024 04:25,19h,2 stops,No info,13882
3,IndiGo,12-05-2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01-03-2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,09-04-2019,Kolkata,Banglore,CCU ? BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27-04-2019,Kolkata,Banglore,CCU ? BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27-04-2019,Banglore,Delhi,BLR ? DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01-03-2019,Banglore,New Delhi,BLR ? DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [131]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset containes 10,683 rows and 11 features.
- Columns 'Rout' and 'Total_Stops' has missing values.
- There are some features wont be treated as datetime data type.

In [132]:
flights.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

## 3. Preliminary Analysis

### 3.1 Check Data Types

In [133]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24-03-2019,Banglore,New Delhi,BLR ? DEL,22:20,22-03-2024 01:10,2h 50m,non-stop,No info,3897
1,Air India,01-05-2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,09-06-2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,10-06-2024 04:25,19h,2 stops,No info,13882
3,IndiGo,12-05-2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01-03-2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [134]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [135]:
value = flights.Date_of_Journey.iloc[6]
value

'12-03-2019'

In [136]:
value = flights.Dep_Time.iloc[6]
value

'18:55'

In [137]:
value = flights.Duration.iloc[6]
value

'15h 30m'

### 3.2 Check for Duplicates

In [138]:
flights.duplicated().sum()

220

In [139]:
# lets check the duplicate data by applying filter

flights[flights.duplicated(keep=False) == True].sort_values(['Airline','Date_of_Journey','Source','Destination'])

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01-03-2019,Banglore,New Delhi,BLR ? BOM ? AMD ? DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
9848,Air India,01-03-2019,Banglore,New Delhi,BLR ? BOM ? AMD ? DEL,08:50,02-03-2024 23:55,39h 5m,2 stops,No info,17135
1495,Air India,01-04-2019,Kolkata,Banglore,CCU ? DEL ? COK ? BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
9913,Air India,01-04-2019,Kolkata,Banglore,CCU ? DEL ? COK ? BLR,10:00,02-04-2024 01:20,15h 20m,2 stops,No info,10408
3598,Air India,01-05-2019,Kolkata,Banglore,CCU ? GAU ? DEL ? BLR,09:50,02-05-2024 08:55,23h 5m,2 stops,No info,13227
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24-03-2019,Banglore,New Delhi,BLR ? DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24-03-2019,Banglore,New Delhi,BLR ? DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24-03-2019,Banglore,New Delhi,BLR ? DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24-03-2019,Banglore,New Delhi,BLR ? DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### 3.3 Observations

>- The datatype of feature 'Date_of_Journey' should be datetime.
>- Same we get stuff at feature 'Dep_Time','Arrival_time'.
>- The type of 'duration' is mixed so we have to handle them.
>- 'Total_Stops' is also mixed and we can convert them to int or float.
>- There are 220 duplicatse this should be removed.

## 4. Detailed Analysis

### Airline

In [140]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [141]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [142]:
# We are putting block of code inside bracket beacuse that will makes better to read.
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
)

0             Indigo
1          Air India
2        Jet Airways
3             Indigo
4             Indigo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

>- Some of the entries has inconsisent values so we need to replace those values.

### Date Of Journey

In [143]:
flights.Date_of_Journey

0        24-03-2019
1        01-05-2019
2        09-06-2019
3        12-05-2019
4        01-03-2019
            ...    
10678    09-04-2019
10679    27-04-2019
10680    27-04-2019
10681    01-03-2019
10682    09-05-2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [144]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source 

In [145]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [146]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### Departure time

In [147]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [148]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [149]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival Time

In [150]:
flights.Arrival_Time

0        22-03-2024 01:10
1                   13:15
2        10-06-2024 04:25
3                   23:30
4                   21:35
               ...       
10678               22:25
10679               23:20
10680               11:20
10681               14:10
10682               19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [157]:
(
    flights
    .Arrival_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ",n=0)
    .str.get(1)
)

0        01:10
2        04:25
6        10:25
7        05:05
8        10:25
         ...  
10666    19:00
10667    20:20
10672    19:00
10673    04:25
10674    21:20
Name: Arrival_Time, Length: 4335, dtype: object

In [161]:
(
    flights
    .Arrival_Time
    .apply(lambda x: x.split(" ")[1] if len(x.split(" ")) > 1 else x)
)

0        01:10
1        13:15
2        04:25
3        23:30
4        21:35
         ...  
10678    22:25
10679    23:20
10680    11:20
10681    14:10
10682    19:15
Name: Arrival_Time, Length: 10683, dtype: object

### Duration

In [169]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [171]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("m")]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [173]:
(
    flights
    .Duration
    .loc[lambda ser: ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [175]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,06-03-2019,Mumbai,Hyderabad,BOM ? GOI ? PNQ ? HYD,16:50,16:55,5m,2 stops,No info,17327


>- This observation indexed is clearly wrong so we can delete this row beacause as we know that there is no flight duration 5m

In [193]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(['hour','minute'], axis=1) # where used to give an columns names
    .assign(
        hour= lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute= lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [199]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(['hour','minute'], axis=1) # where used to give an columns names
    .assign(
        hour= lambda df_:(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute= lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()    
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


### TotalStops

In [211]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [225]:
(
    flights
    .Total_Stops
    .replace("non-stop","0")
    .str.replace(" stops?","", regex =True) # This is an regular expression that ? helps t search for pattern stop or stops so that s is an optional
    .pipe(lambda ser: pd.to_numeric(ser)) 
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

### Addition Info

In [236]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5.Cleaning Opertaions

In [205]:
def convert_to_minutes(ser):
    return(
        ser
        .str.split(" ", expand=True)
        .set_axis(['hour','minute'], axis=1) # where used to give an columns names
        .assign(
            hour= lambda df_:(
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minute= lambda df_:(
                df_
                .minute
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
        )
        .sum(axis=1)
    )

In [252]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col: df[col].str.strip()
            for col in df.select_dtypes(include='O').columns
        })
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_: (
                df_
                .airline
                .str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey=lambda df_: pd.to_datetime(df_.date_of_journey, dayfirst=True),
            dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_: (
                df_
                .arrival_time
                .apply(lambda x: x.split(" ")[1] if len(x.split(" ")) > 1 else x)
            ),
            duration= lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_:(
                df_
                .total_stops
                .replace("non-stop","0")
                .str.replace(" stops?","", regex =True) # This is an regular expression that ? helps t search for pattern stop or stops so that s is an optional
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_: df_.additional_info.replace("No info","No Info")
        )
        .drop(columns='route')
    )

>- pipe where used in pandas to apply fun over series or dataframe

In [253]:
flights_cleaned = clean_data(flights)
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10,160,0.0,No Info,12648


## 6. Split the Data

In [254]:
flights_final = flights_cleaned.sample(1000)
flights_final

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
2054,Air India,2019-06-06,Mumbai,Hyderabad,06:20:00,07:40,80,0.0,No Info,3100
7758,Air India,2019-05-15,Kolkata,Banglore,09:50:00,23:15,805,2.0,No Info,14960
10327,Spicejet,2019-04-12,Kolkata,Banglore,06:55:00,09:30,155,0.0,No check-in baggage included,3841
5081,Jet Airways,2019-06-12,Kolkata,Banglore,06:30:00,19:50,800,1.0,No Info,13626
1180,Jet Airways,2019-06-09,Delhi,Cochin,06:45:00,04:25,1300,2.0,No Info,13376
...,...,...,...,...,...,...,...,...,...,...
3758,Jet Airways,2019-05-01,Mumbai,Hyderabad,02:55:00,04:20,85,0.0,In-flight meal not included,3210
4319,Indigo,2019-03-09,Chennai,Kolkata,05:15:00,07:35,140,0.0,No Info,6482
6743,Air India,2019-03-27,Delhi,Cochin,06:10:00,08:15,1565,1.0,No Info,6681
6823,Jet Airways,2019-03-06,Banglore,New Delhi,05:45:00,19:10,805,1.0,No Info,16736


In [255]:
X = flights_final.drop(columns="price")
y = flights_final.price.copy()

In [256]:
X_,X_test,y_,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_,y_,test_size=0.2,random_state=42)

In [257]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## 7. Export the Subsets

In [258]:
def export_subset(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(Project_Dir, Data_Dir, file_name)
    
    X.join(y).to_csv(file_path, index=False)
    
    return pd.read_csv(file_path).head()

In [259]:
export_subset(X_train, y_train, 'train')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-06-06,Delhi,Cochin,14:20:00,22:30,490,1.0,No Info,6938
1,Multiple Carriers,2019-06-15,Delhi,Cochin,11:30:00,19:15,465,1.0,No Info,16108
2,Jet Airways,2019-05-12,Kolkata,Banglore,17:00:00,22:05,1745,1.0,No Info,13067
3,Air India,2019-05-09,Delhi,Cochin,14:25:00,19:15,1730,2.0,No Info,13591
4,Air India,2019-03-06,Mumbai,Hyderabad,12:45:00,19:25,1840,2.0,No Info,13253


In [260]:
export_subset(X_test, y_test , 'test')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-06-09,Kolkata,Banglore,06:30:00,04:40,1330,1.0,In-flight meal not included,9663
1,Jet Airways,2019-04-09,Banglore,Delhi,15:50:00,18:45,175,0.0,No Info,7229
2,Jet Airways,2019-05-27,Delhi,Cochin,05:30:00,12:35,425,2.0,In-flight meal not included,15544
3,Jet Airways,2019-03-03,Delhi,Cochin,16:00:00,18:50,1610,1.0,No Info,17234
4,Indigo,2019-06-27,Delhi,Cochin,21:50:00,03:35,345,1.0,No Info,5775


In [261]:
export_subset(X_val, y_val , 'validation')

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-05-09,Delhi,Cochin,18:15:00,12:35,1100,1.0,In-flight meal not included,12373
1,Jet Airways,2019-05-03,Mumbai,Hyderabad,07:05:00,08:30,85,0.0,In-flight meal not included,4995
2,Jet Airways,2019-05-18,Delhi,Cochin,02:15:00,04:25,1570,1.0,In-flight meal not included,12373
3,Indigo,2019-05-15,Chennai,Kolkata,22:05:00,00:25,140,0.0,No Info,5277
4,Multiple Carriers,2019-05-15,Delhi,Cochin,14:00:00,01:30,690,1.0,No Info,13727
